In [22]:
import sys
import os
import numpy as np

sys.path.append(os.path.abspath(".."))
from src.config import Config
from src.backtest import Backtester
from src.backtest_min import BacktesterMin

config = Config()

In [11]:
TOKEN = config.get("TINKOFF_TOKEN")  # берём токен из переменной окружения

In [23]:
from tinkoff.invest import Client
import os

with Client(TOKEN) as client:
    instruments = client.instruments.shares().instruments
    for share in instruments:
        if share.ticker == "SBER":  # ищем Сбербанк
            print(share.figi, share.name)
            FIGI = share.figi

BBG004730N88 Сбер Банк


# Backtesting lib

In [55]:
import os
from datetime import datetime, timedelta

import pandas as pd
from tinkoff.invest import Client, CandleInterval
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA


def load_candles(figi: str, days: int = 200, interval=CandleInterval.CANDLE_INTERVAL_DAY) -> pd.DataFrame:
    """
    Загружаем свечи с Tinkoff Invest API и возвращаем DataFrame для backtesting.py
    """
    end = datetime.utcnow()
    start = end - timedelta(days=days)

    with Client(TOKEN) as client:
        candles = client.market_data.get_candles(
            figi=figi,
            from_=start,
            to=end,
            interval=interval
        )

    data = []
    for c in candles.candles:
        data.append({
            "Date": c.time,
            "Open": c.open.units + c.open.nano / 1e9,
            "High": c.high.units + c.high.nano / 1e9,
            "Low": c.low.units + c.low.nano / 1e9,
            "Close": c.close.units + c.close.nano / 1e9,
            "Volume": c.volume,
        })

    df = pd.DataFrame(data)
    df = df.sort_values("Date").reset_index(drop=True)
    df.set_index("Date", inplace=True)  # backtesting.py требует индекс по времени
    return df



In [77]:
df = load_candles(FIGI, days=365 * 5, interval=CandleInterval.CANDLE_INTERVAL_DAY)
print(df.head())

/var/folders/72/b7x6pblx55l7d1bw9rtw10fc0000gq/T/ipykernel_17111/4236298551.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = datetime.utcnow()


                             Open    High     Low   Close    Volume
Date                                                               
2020-09-11 00:00:00+00:00  221.46  223.15  219.75  220.53  45688920
2020-09-14 00:00:00+00:00  222.31  227.54  222.13  227.07  62906200
2020-09-15 00:00:00+00:00  228.22  231.95  227.39  231.30  61902240
2020-09-16 00:00:00+00:00  231.72  232.60  230.15  231.35  39605930
2020-09-17 00:00:00+00:00  229.10  231.75  228.57  231.75  49126710


In [78]:
class SmaCross(Strategy):
    fast = 20
    slow = 100

    def init(self):
        close = self.data.Close
        self.sma_fast = self.I(SMA, close, self.fast)
        self.sma_slow = self.I(SMA, close, self.slow)

    def next(self):
        if crossover(self.sma_fast, self.sma_slow):
            self.buy(size=0.8)
        elif crossover(self.sma_slow, self.sma_fast):
            self.sell(size=0.8)

        # if crossover(self.sma_fast, self.sma_slow):
        #     self.buy(size=10)   # вместо "всё депо"
        # elif crossover(self.sma_slow, self.sma_fast):
        #     self.sell(size=10)


In [79]:
class RiskManagedSmaCross(Strategy):
    fast = 50
    slow = 100
    risk_per_trade = 0.01  # 1% капитала

    def init(self):
        close = self.data.Close
        self.sma_fast = self.I(SMA, close, self.fast)
        self.sma_slow = self.I(SMA, close, self.slow)

    def next(self):
        price = self.data.Close[-1]

        # Закрываем позицию, если SMA пересеклись в другую сторону
        if crossover(self.sma_slow, self.sma_fast):
            self.position.close()

        # Вход в лонг
        elif crossover(self.sma_fast, self.sma_slow):
            if not self.position:  # только если позиции нет
                # стоп-лосс под локальным минимумом (или на 2% ниже цены)
                stop = price * 0.98  
                risk_amount = self.equity * self.risk_per_trade
                trade_risk = price - stop

                # размер позиции = сколько акций купить при риске 1%
                size = risk_amount // trade_risk  

                if size > 0:
                    self.buy(size=size, sl=stop)

In [81]:
df

,Open,High,Low,Close,Volume
2020-09-11 00:00:00+00:00,221.46,223.15,219.75,220.53,45688920
2020-09-14 00:00:00+00:00,222.31,227.54,222.13,227.07,62906200
2020-09-15 00:00:00+00:00,228.22,231.95,227.39,231.30,61902240
2020-09-16 00:00:00+00:00,231.72,232.60,230.15,231.35,39605930
2020-09-17 00:00:00+00:00,229.10,231.75,228.57,231.75,49126710
...,...,...,...,...,...
2025-09-06 00:00:00+00:00,310.50,310.90,310.10,310.13,654610
2025-09-07 00:00:00+00:00,310.13,311.00,310.13,310.69,734383
2025-09-08 00:00:00+00:00,310.44,313.26,310.31,312.83,16963643
2025-09-09 00:00:00+00:00,313.00,314.25,311.62,313.06,17116843


In [80]:
bt = Backtest(df, SmaCross, cash=1_000_000, commission=.0004)
stats = bt.run()
print(stats)
bt.plot()

Backtest.run:   0%|          | 0/1194 [00:00<?, ?bar/s]

Start                     2020-09-11 00:00...
End                       2025-09-10 00:00...
Duration                   1825 days 00:00:00
Exposure Time [%]                    75.50232
Equity Final [$]                2196300.33374
Equity Peak [$]                  2307573.9856
Commissions [$]                    1639.98639
Return [%]                          119.63003
Buy & Hold Return [%]                18.99446
Return (Ann.) [%]                    16.55818
Volatility (Ann.) [%]                20.24871
CAGR [%]                             11.47604
Sharpe Ratio                          0.81774
Sortino Ratio                         1.67195
Calmar Ratio                          0.64316
Alpha [%]                           120.04983
Beta                                  -0.0221
Max. Drawdown [%]                   -25.74484
Avg. Drawdown [%]                     -3.1083
Max. Drawdown Duration      326 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                          

/var/folders/72/b7x6pblx55l7d1bw9rtw10fc0000gq/T/ipykernel_17111/1841235001.py:2: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:709: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:709: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p5571', ...)